# Введение

В этой лекции мы поговорим о ...

# Алгоритм k-nearest neighbors



Продолжим с того места, где мы остановились в предыдущей лекции - классификация методом ближайших соседей (kNN). 

**Метрики расстояния**

В практических заданиях EX01 мы использовали kNN для классификации изображений CIFAR10 (набор фотографий разделенных на 10 классов). Мы выяснили что точность метода kNN с использованем расстояний L1 (*Manhatten distance* - сумма абсолютных разностей между пикселями) оставляет желать лучшего, и попробовали применить L2 (*Euclidian distance*) в надежде эту точность классификации повысить. С этими метриками мы будем сталкиваться часто: и в качестве loss функции, и в качестве регуляризации, поэтому познакомиться с ними полезно. 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_Classifier-1.png" width="600">

## KNN для классификации



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier-KNN.jpg" width="450">

На практике, метод ближайших соседей для классификации используется крайне редко. Давайте разберемся почему.
Проблема заключается в следующем: предположим, что точность классификации нас устраивает. Теперь давайте применим kNN для по больших данных (e.g. миллион картинок). Для определения класса каждой из картинок, нам нужно сравнить ее со всеми другими картинками в базе данных, такие расчеты, даже в супер оптимизированном виде, занимают много времени. Очень много времени.  Мы хотим, чтобы обученная модель работала быстро.

Тем не менее, метод ближайших соседей используется в других задачах, где без него обойтись сложно. Например, в задаче распознавания лиц. Представим, что у нас у нас есть большая база данных с фотографиями лиц (например по 5 разных фотографий всех сотрудников, которые работают в офисном здании, как на примере выше) и есть камера установленная на входе в это здание. Мы хотим узнать кто и во сколько пришел на работу. Для того, что бы понять кто прошел перед камерой нам нужно зафиксировать лицо этого человека и сравнить его со всем фотографиями лиц в базе. В такой формулировке мы не пытаемся определить конкретный класс фотографии, а всего лишь определяем “похож-не похож”. Мы смотрим на k ближайших соседей и, например, если из k соседей 5 - это фотографии Джеки Чана, то скорее всего, под камерой прошел именно он. В таких случаях kNN метод вполне полезен. Похожим образом работает и поиск дупликатов в базах данных.

Примеры эффективной реализации метода на основе kNN:
* [Facebook AI Research Similarity Search](https://github.com/facebookresearch/faiss) – разработка команды Facebook AI Research для быстрого поиска ближайших соседей и кластеризации в векторном пространстве. Высокая скорость поиска позволяет работать с очень большими данными – до нескольких миллиардов векторов.
* Алгоритм поиска ближайших соседей [Hierarchical Navigable Small World](https://arxiv.org/abs/1603.09320). 

## Практические аспекты работы с классификаторами

### Переобучение

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков, соотвествует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30 признакам обучить модель определять злокачественная или доброкачественная ткань.

Можно иметь сколь угодно хороший алгоритм для классификации - но до тех пор, пока данные на входе - мусор, на выходе из нашего чудесного классификатора мы тоже будем получать мусор (*garbage in - garbage out*). Давайте разберемся что конкретно надо сделать, что бы kNN реально заработал.


In [ ]:
# импортируем нужные библиотеки
import sklearn.datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [ ]:
cancer = sklearn.datasets.load_breast_cancer() # Загружаем датасет
X = cancer.data # Здесь будем хранить признаки
y = cancer.target # Здесь храним лейблы(классы)
print(f'Размерность X {X.shape}, Размерность y {y.shape}') # Посмотри на размерность датасета
print(f'Пример X \n {X[0]}') # Посмотрим как выглядит набор признаков для одного примера
print(f'Пример y \n {y[0]}') # Посмотрим как выглядит пример лейбла

Посмотрим сколько данных в классе `0` и сколько данных в классе `1`

In [ ]:
# plt.style.use('default') # Установим сетку для отрисовки графика
plt.figure(figsize=(8,5)) # Установим размер графика
plt.bar(1,y[y==1].shape, label=cancer.target_names[0]) # Строим первый столбец 
plt.bar(0,y[y==0].shape, label=cancer.target_names[1]) # Строим второй столбец
plt.title('Баланс классов') # Название графика
plt.ylabel('Количество в каждом классе') # Название оси Y
plt.xticks(ticks=[1,0], labels=['1','0']) # Название значение по оси X
plt.legend(loc='upper left') # Отображение легенды, в левом верхнем угле
plt.show() # Отображаем график

Теперь давайте посмотрим на сами данные. У нас есть 569 строк в каждой из которой по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд)

In [ ]:
pd.DataFrame(X).describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные значения

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(X), orient="h", palette="Set2")
ax.set(xscale='log', xlim=(1e-4, 1e4))

Что бы адекватно сравнить данные между собой нам следует использовать нормализацию.

 **Нормализация, выбор Scaler**

Нормализация — это преобразование данных к неким безразмерным единицам.
Ключевая цель нормализации — приведение различных данных в самых разных единицах измерения и диапазонах значений к единому виду, который позволит сравнивать их между собой или использовать для расчёта схожести объектов.
Итого, главное условие правильной нормализации — все признаки должны быть равны в возможностях своего влияния.

Например, у нас данные по группе людей возраст (в годах) и размер дохода (в рублях). Возраст может измениться в диапозоне от 18 до 70 ( интервал 70-18 = 52). А доход от 30 000 р до 500 000 р (интервал 500 000 - 30 000 = 470 000). В таком варианте разница в возрасте имеет меньшее влияние, чем разница в доходе. Получается, что доход становится более важным признаком, изменения в котором влияют больше при сравнении схожести двух людей.

Должно быть так, чтобы максимальные изменения любого признака в «основной массе объектов» были одинаковы. Тогда потенциально все признаки будут равноценны.
Точно степень влияния признака должно определить только обучение модели. 

Осталось определиться с выбором инструмента, часто используют следующие варианты: `MinMaxScaler`, `StandardScaler`, `RobustScaler`

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные в диапозоне от 0 до 1. Может быть полезно, если вы хотите выполнить преобразование, где отрицательные значения не допускаются (e.g., масштабирование RGB пикселей)


$$z=\frac{X_i-X_{min}}{X_{max}-X_{min}}$$

$X_{min}$ и $X_{max}$ задаются как минимальное и максимальное допустимое значение, по умолчанию:  $X_{min}=0$  и $X_{max}=1$

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение 0 и стандартное отклонение 1. Большинство значений будет в  диапозоне от -1 до 1. Это стандартная трансформация, и она применима во многих ситуациях

$$z=\frac{X-u}{s}$$

$u$ — среднее значение (или 0 при `with_mean=False`) и $s$ — стандартное отклонение (или 0 при `with_std=False`)

И StandardScaler и MinMaxScaler очень чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях* (процентиль — мера, в которой процентное значение общих значений равно этой мере или меньше ее. Например, 90 % значений данных находятся ниже 90-го процентиля, а 10 % значений данных находятся ниже 10-го процентиля) и поэтому не зависит от небольшого числа очень больших предельных выбросов (*outliers*). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков

$$z=\frac{X-X_{median}}{IQR}$$

$X_{median}$ — значение медианы, $IQR$ — межквартильный диапазон равный разнице между 75-ым и 25-ым процентилями

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для признака data[:,0]. Обратите внимание на ось X

In [ ]:
np.random.seed(42)  # задаем параметр инициализации случайных значений

# генерируем случайные значения от 1 до 255 размерностью (30,1)
test = X[:,0].reshape(-1,1)

plt.figure(1, figsize=(30, 5))  # задаем размер графика
plt.subplot(141)  # указываем расположение графика
plt.scatter(test, range(len(test)), c=y)  # строим график точек scatter
plt.ylabel("Количество точек", fontsize=15)  # указываем название оси Y
plt.xticks(fontsize=15)  # указываем размер шрифта по оси Х для значений
plt.yticks(fontsize=15)  # указываем размер шрифта по оси Y для значений
plt.title("Необработанные данные", fontsize=18)  # указываем название графика

# масштабируем значения с помощью MinMaxScaler
test_scaled = MinMaxScaler().fit_transform(test)  
plt.subplot(142)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("После MinMaxScaler", fontsize=18)

# масштабируем значения с помощью StandardScaler
test_scaled = StandardScaler().fit_transform(test)  
plt.subplot(143)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("После StandardScaler", fontsize=18)

# масштабируем значения с помощью RobustScaler
test_scaled = RobustScaler().fit_transform(test)  
plt.subplot(144)
plt.scatter(test_scaled, range(len(test)), c=y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("После RobustScaler", fontsize=18)
plt.show()

Для нашей задачи по определению раковых опухолей обработаем наши 30 признаков с помощью StandardScaler.

In [ ]:
X_norm = StandardScaler().fit_transform(X)  # масштабирум данные

Видим что они стали намного более сравнимы между собой

In [ ]:
pd.DataFrame(X_norm).describe()

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(X_norm), orient="h", palette="Set2")

Теперь обучим KNN для общей выборки данных, при разном значении количества соседей

In [ ]:
n_nei_rng = np.arange(1, 31)  # создаем массив из количества соседей

# создаем массив из нулей размерностью как массив количества соседей
quality = np.zeros(
    n_nei_rng.shape[0]
)  

for ind in range(n_nei_rng.shape[0]):  # проходимся по по индексам массивов
    # создаем knn для разного количества соседей
    knn = KNeighborsClassifier(
        n_neighbors=n_nei_rng[ind]
    )  
    knn.fit(X_norm, y)  # обучаем knn
    q = accuracy_score(y_pred=knn.predict(X_norm), y_true=y)  # считаем точность
    quality[ind] = q  # заполняем массив значениями точности

plt.figure(figsize=(8, 5))  # задаем размеры графика
plt.title("KNN on train", size=20)  # задаем название графика
plt.xlabel("Neighbors", size=15)  # задаем название оси X
plt.ylabel("Accuracy", size=15)  # задаем название оси Y
plt.plot(n_nei_rng, quality)  # строим график
plt.xticks(n_nei_rng)  # указываем метки для оси X
plt.show()  # отображаем график

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты.

Если теперь мы попробуем взять какой-то новый образец опухоли и классифицировать его - у нас скореее всего ничего не получится. В таких случаях мы говорим что наша модель не умеет обобщать (*generalization*).

Для того, чтобы знать заранее обобщает ли наша модель или нет, мы можем разбить все имеющиеся у нас данныe на 2 части. Но одной части мы будем обучать классификатор (*train set*), а на другой тестировать насколько хорошо он работает (*test set*).

In [ ]:
from sklearn.model_selection import train_test_split

# делим выборки на обучающую и поверочную
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)  
scaler = StandardScaler()  # создаем scaler
scaler.fit(X_train)  # обучаем scaler
X_train_norm = scaler.transform(X_train)  # масштабируем данные
X_test_norm = scaler.transform(X_test)  # масштабируем данные

n_nei_rng = np.arange(1, 31)  # создаем массив количества соседей от 1 до 30
train_quality = np.zeros(n_nei_rng.shape[0])  # качество модели на обучающей выборке
test_quality = np.zeros(n_nei_rng.shape[0])  # качество модели на поверочной выборке

for ind in range(n_nei_rng.shape[0]):  # проходимся по всем индексам массива
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])  # создаем KNN
    knn.fit(X_train_norm, y_train)  # обучаем KNN
    
    # считаем точность на обучающей выборке
    trq = accuracy_score(
        y_pred=knn.predict(X_train_norm), y_true=y_train
    )  
    train_quality[ind] = trq  # добавляем в массив

    # считаем точность на поверочной выборке
    teq = accuracy_score(
        y_pred=knn.predict(X_test_norm), y_true=y_test
    )  
    test_quality[ind] = teq  # добавляем в массив

# строим график сравнения работы модели на обучающей и поверочной выборке
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой". Такие случаи мы называем *переобучением*. Что бы действительно предсказывать что-то полезное нам надо выбирать число соседей начиная минимум с 3

### Кросс-валидация

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель менять параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. число соседей 
2. функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. веса, с которыми складываем метки ближайших соседей
4. признаки! (но об этом с вами поговорим позже)
5. сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выборов просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что классификация такими моделями ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(y_real):
    # создаем массив из значений True/False, для случайного выбора значений по маске
    guessed = np.random.choice(
        [
            True,
            False,
        ],  
        size=y_real.shape[0],
        replace=True,
    )
    y_predicted = np.zeros_like(y_real)  # создаем массив из нулей размером y_real
    
    # с помощью маски guessed присваиваем значения y_predicted
    y_predicted[guessed] = y_real[guessed]  
    y_predicted[~guessed] = 1 - y_real[~guessed]  # меняем значения по обратной маске
    return y_predicted

In [ ]:
models_num = 10000  # количество экспериментов
best_quality = 0.5  # порог качества по точности

# создаем случайный массив из значений от 0 до 1.
y_real = np.random.choice(
    [0, 1], size=250, replace=True
)  

for i in range(models_num):  # запускаем в цикле количество экспериментов
    y_pred = guess_model(y_real)  # создаем с помощью функции "предсказанные" значения
    q = accuracy_score(y_pred=y_pred, y_true=y_real)  # считаем точность
    if q > best_quality:  # если точность больше порога
        best_quality = q  # то порог становится текущим максимальным значением точности
print(f"Лучший результат {best_quality}")

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1) подбираем гиперпараметры моделей на отдельном датасете, называмым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-cross-validation.png" width="400">

2) чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - кросс-валидация

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier-cross-validation2.png" width="500">

Часто применяется следующий подход, называемый [K-Fold кросс-валидацией](https://scikit-learn.org/stable/modules/cross_validation.html):

Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, надо попробовать другое К и если ни при каком не получилось - то это такие данные.  


Подберем параметры для модели с помощью GridSearchCV

GridSearchCV – это инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

"""
Параметры для GridSearchCV:

estimator — модель которую хотим обучать (алгоритм)

cv — сколько разрезов кросс-валидации мы ходим сделать

param_grid — передаем какие параметры хотим подбирать, GridSearchCV на всех параметрах сделает обучение

scoring — выбор метрики ошибки (для разных задач можно выбрать разные функции ошибки)

n_jobs - количество процессов выполняемых параллельно, значение -1, задействовать все возможные
"""
model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=KFold(3, shuffle=True, random_state=42),
    param_grid={
        "n_neighbors": np.arange(1, 31),
        "metric": ["euclidean", "manhattan"],
        "weights": ["uniform", "distance"],
    },
    scoring="accuracy",
    n_jobs=-1,
)
model.fit(X_train_norm, y_train)

Выведем лучшие гиперпараметры для модели, которые подобрали:

In [ ]:
print("Метрика:", model.best_params_["metric"])
print("Количество соседей:", model.best_params_["n_neighbors"])
print("Веса:", model.best_params_["weights"])


Объект GridSearchCV можно использовать как обычную модель

In [ ]:
y_pred = model.predict(X_test_norm)
print(f"Процент правильных ответов {np.round(accuracy_score(y_pred=y_pred, y_true=y_test)*100,2)} %")
print(f"Процент правильных ответов по отношению к классам {np.round(balanced_accuracy_score(y_pred=y_pred, y_true=y_test)*100,2)} %")

Мы можем извлечь дополнительные данные о кроссвалидации и по ключу обратиться к результатам всех моделей

In [ ]:
list(model.cv_results_.keys())

Выведем для примера mean_test_score:

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(model.cv_results_["mean_test_score"])
plt.title("mean_test_score", size=20)
plt.xlabel("Количество экспериметов", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Построим, например, при фиксированных остальных параметрах (равных лучшим параметрам), качество модели на валидации в зависимости от числа соседей

In [ ]:
selected_means = []
selected_std = []
n_nei = []
for ind, params in enumerate(model.cv_results_["params"]):
    if (
        params["metric"] == model.best_params_["metric"]
        and params["weights"] == model.best_params_["weights"]
    ):
        n_nei.append(params["n_neighbors"])
        selected_means.append(model.cv_results_["mean_test_score"][ind])
        selected_std.append(model.cv_results_["std_test_score"][ind])

Построим error bar, для сравнения разброса ошибки при разном количестве соседей Neighbors

In [ ]:
plt.figure(figsize=(8, 5))
plt.title(f"KNN CV, {params['metric']}, {params['weights']}", size=20)
plt.errorbar(n_nei, selected_means, yerr=selected_std, linestyle="None", fmt="-o")
plt.xticks(n_nei)
plt.ylabel("Mean_test_score", size=15)
plt.xlabel("Neighbors", size=15)

plt.show()

Видим, что на самом деле большой разницы в числе соседей и нет. 

**Можно ли делать только кросс-валидацию (без теста)?**


Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится. Оценка конечного качества модели должно производиться на отложенной тестовой выборке. Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://weina.me/nested-cross-validation/), но об этом позже в последующих лекциях. Но даже в этом случае придется анализировать поведение модели, чтобы показать, что она учит что-то разумное. Кстати, вложенную кросс-валидацию можно использовать чтобы просто получить более устойчивю оценку поведения модели на тесте.

# Линейный классификатор

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-04.jpg" width="800">

Давайте подумаем, как избавиться от проблемой со скоростью K-Nearest Neighbors. Реализуя метод ближайшего соседа, мы сравнивали одно изображение со всеми, при этом мы предполагали, что изображения из одного класса будут чем-то похожи друг на друга, и поэтому разница будет меньше. Метод как-то работал. 

Чтобы ускорить этот процесс, мы можем взять весь блок изображений (справа), который относится, например, к машинам и усредним. Таким образом, получим некоторый шаблон для класса “автомобиль”. Скорее всего, работать такой подход будет не слишком здорово, но зато вместо тысяч изображений (в данном случае 50000) появится одно. Возможно, это поможет сильно сэкономить время.

## Переход к сравнению с шаблоном

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-05.jpg" width="450">

Идея в следующем: вместо того, чтобы сравнивать каждое изображение со всеми остальными по очереди, будем сравнивать изображения с шаблоном для каждого класса. Их будет всего 10 для данного датасета. Этот подход позволит радикально увеличить скорость. 

Проверим, что получится из этой идеи. 


скачиваем CIFAR10 в архиве

In [ ]:
from IPython.display import clear_output 
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz
!ls -l
clear_output()

Загрузили архив в пямять целиком, используя для этого фрагмент кода с сайта CIFAR10. 

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

path = '/content/cifar-10-batches-py/'

x_train = np.zeros((0, 3072))
y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"{path}/data_batch_{i}")
    x_train = np.append(x_train, np.array(raw[b"data"]), axis=0)
    y_train = np.append(y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle(f"{path}/test_batch")
x_test = np.array(test[b"data"])
y_test = np.array(test[b"labels"])

labels_ru = [
    "Самолет",
    "Автомобиль",
    "Птица",
    "Кошка",
    "Олень",
    "Собака",
    "Лягушка",
    "Лошадь",
    "Корабль",
    "Грузовик",
]

print(f"Размерность обучающей выборки X: {x_train.shape}, y: {y_train.shape}")
print(f"Размерность тестовой выборки X: {x_test.shape}, y: {y_test.shape}")

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import pylab

pylab.rcParams["figure.figsize"] = (20.0, 20.0)  # задаем размеры графика
for i in range(10):  # проходим по классам от 0 до 9
    # получаем список из индексов положений класса i в y_train
    label_indexes = np.where(y_train == i)[0]  
    index = np.random.choice(label_indexes)  # случайным образом выбираем из списка индекс
    # Выбираем из x_train нужное изображение
    img = x_train[index].reshape(
        32, 32, 3, order="F"
    )  
    img = np.rot90(img, k=3)  # поворачиваем изображение
    plt.subplot(1, 10, i + 1)  # строим 10 изображений
    plt.title(labels_ru[i])  # указываем название каждого изображения по классу
    plt.axis("off")  # отключаем отображение осей
    imshow(img.astype("uint8"))  # отрисовываем изображения

Для начала создадим шаблоны и визуализируем их.


In [ ]:
templates = []  # здесь будем хранить шаблоны
meta = unpickle(f"{path}/batches.meta")  # загружаем данные
labels = meta[b"label_names"]  # берем названия лейблов

for i in range(len(labels)):
    indexes = np.where(y_train == i)  # получаем индекс для каждого класса
    mask = np.zeros(len(y_train), dtype=bool)  # создаем маску из значений False
    mask[indexes, ] = True  # меняем значения на True по индексам
    images = x_train[mask]  # выбираем все изображения одного класса
    mn = np.mean(images, 0)  # считаем среднее
    templates.append(mn)

In [ ]:
plt.figure(figsize = (20.0, 2.0))

def show_templates(templates, labels):
    for i, template in enumerate(templates):
        img = template.reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
        plt.subplot(1, len(labels), i + 1)
        plt.title(labels_ru[i])
        plt.axis("off")
        imshow(img)

show_templates(templates, labels)  # отрисовывем получившиеся шаблоны

Напишем классификатор, который будет сравнивать изображение с шаблоном, который генерируется во время обучения.

In [ ]:
class TemplateBasedClassifier:
    def __init__(self):
        self.templates = []  # здесь будем хранить шаблоны
        meta = unpickle(f"{path}/batches.meta")  # загружаем данные
        self.labels = meta[b"label_names"]  # берем названия лейблов

    def fit(self, x_train, y_train):
        self.templates = []
        for label_num in range(len(self.labels)):  # проходимся по всем классам
            # получаем индекс для каждого класса
            indexes = np.where(y_train == label_num)  
            mask = np.zeros(len(y_train), dtype=bool)  # создаем маску из значений False
            mask[indexes, ] = True  # меняем значения на True по индексам
            images = x_train[mask]  # выбираем все изображения одного класса
            mn = np.mean(images, 0)  # считаем среднее значение пикселей для каждого изображения класса
            self.templates.append(mn)  # добавляем в шаблоны

    def forward(self, x):
        distances = np.sum(
            np.abs(self.templates - x), axis=1)  # отнимаем от шаблонов значения изображения, считаем разницу
        return np.argmin(distances)  # возвращаем индекс минимального значения

Теперь у нас есть 10 шаблонов, с которыми мы будем сравнивать изображение и на основании этого делать предсказание. На этих шаблонах можно увидеть очертания объектов.

Рассмотрим сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние L1 между ними. Чем желтее цвет - тем больше изображени похоже на шаблон

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3, figsize=(10,5))

indexes = np.where(y_train == 3)  # получаем индекс для каждого класса
mask = np.zeros(len(y_train), dtype=bool)  # создаем маску из значений False
mask[indexes, ] = True  # меняем значения на True по индексам
images = x_train[mask]

img = images[0].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
template = templates[3].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
residual = np.mean(np.abs(img-template),-1)
ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='L1')

ax[0].set_title('Изображение')
ax[1].set_title('Шаблон')
ax[2].set_title('D = Изображение - Шаблон')

for a in ax:
    a.axis('off')

Сделаем предсказание и замерим время

In [ ]:
model = TemplateBasedClassifier()  # создаем наш классификатор
model.fit(x_train, y_train)  # запускаем обучение

In [ ]:
import time

def validate(model, x_test, y_test, noprint=False):
    correct = 0  # здесь храним количество верных предсказаний
    for i, img in enumerate(x_test):  # проходимся по всем изображениям в выборке
        index = model.forward(img)  # определяем класс изображения
        correct += (
            1 if index == y_test[i] else 0
        )  # если класс верный то увеличиваем correct на 1
        if noprint is False:  # если стоит флаг False
            if i > 0 and i % 1000 == 0:  # каждые 1000 значений
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  # выводим значения точност, количество верных ответов/количество попыток
    return correct / len(y_test)  # возвращаем общее значение точности

start = time.perf_counter()  # засекаем время
accuracy= validate(model, x_test, y_test)  # считаем точность
tm = time.perf_counter() - start  # считаем, сколько заняло времени
print(
    "Accuracy {:.2f} Train {:d} /test {:d} in {:.1f} sec. speed {:.2f} samples per second.".format(
        accuracy, len(x_train), len(x_test), tm, len(x_test) / tm
    )
)

Если бы мы обучали KNN на таком количестве данных, это заняло бы у нас 2 часа.



##  Переход к весам
----
Умножение вместо вычитания - для чего?


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-06.jpg" width="450">

В сранении с шаблонами, мы вычитали нашу картинку из шаблона и таким образом смотрели насколько они похожи (L1). Теперь давайте сделаем следующий, пока ничем не обоснованный, ход: оставим всё, как было, но заменим вычитание умножением. Логика в том, что не все пиксели одинаково важны. Вероятно, если на изображении совпадут какие-то пиксели, которые отвечают, например, за глаза кошки, это будет намного важнее, чем фон, который может быть точно таким же у собаки. Здесь будут какие-то важные особенности, которым можно придать больший вес.

Если мы распишем в виде формул наложение шаблона на картинку таким образом с умножением, то получается, что мы скалярно перемножаем два вектора. Подробнее <a href="https://ru.wikipedia.org/wiki/Скалярное_произведение">скалярное произведение</a> векторов. Для того, чтобы получить веткор из изображения размерностью 32х32х3, достаточно "выпрямить" его, получим вектор размерностью 1х3072.


Опять рассмотри сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние между ними используя наш новый метод перемножения. Чем желтее цвет - тем больше изображени похоже на шаблон

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3, figsize=(10,5))

indexes = np.where(y_train == 3)  # получаем индекс для каждого класса
mask = np.zeros(len(y_train), dtype=bool)  # создаем маску из значений False
mask[indexes, ] = True  # меняем значения на True по индексам
images = x_train[mask]

img = images[0].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
template = templates[3].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
residual = np.mean(np.abs(img*template),-1)
ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='Расстояние')

ax[0].set_title('Изображение')
ax[1].set_title('Шаблон')
ax[2].set_title('D = Изображение - Шаблон')

for a in ax:
    a.axis('off')

##  Математическая запись

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-08.jpg" width="700">



Обозначим входное изображение как $x$, а шаблон для первого из классов как $w_0$
Элементы пронумеруем подряд 1,2,3 … $n$
То есть развернем матрицу пикселей изображения в вектор. 



Тогда результат сравнения изображения с этим шаблоном будет вычисляться по формуле: $x[0]*w0[0] + x[1]*w0[1] + … x[n-1]*w0[n-1]$ 




<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-skalyar.png" width="800">


Эта простая модель лежит в основе практически всех сложных, которые мы будем рассматривать дальше. Внутри мы будем также пользоваться скалярным произведением.

В дальнейшем мы будем проходить сверточные сети, они работают очень похоже:
мы тоже накладываем шаблон на некоторую матрицу и перемножаем элементы, затем складываем. Единственное отличие – обычно ядро свертки меньше, чем размер самого изображения. 



## Support Vector Machine (метод опорных векторов)

[Отличное видео про SVM от Stat Quest которое все объясняет](https://www.youtube.com/watch?v=efR1C6CvhmE&ab_channel=StatQuestwithJoshStarmer)

### 1D классификация

Рассмотрим одномерный пример. У нас есть данные по массе мышей. Часть из них определена как нормальные, а часть как мыши с ожирением. Что бы их отдельить друг от друга, нам достаточно одного критерия. Мы можем посмотреть на график, и визуально определить предельную массу, после которой мышки будут жирненькими

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def generate_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), np.random.uniform(24,33, total_len//2)])
    y = np.hstack([np.zeros(total_len//2), np.ones(total_len//2)])
    return X,y

def plot_data(X, y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=y, s=s)
    ax.axvline(threshold, color='red', ls='dashed')

    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Нормальные', 'С ожирением'])
    ax.set(xlabel='Масса, г');
    return ax

total_len = 40
X,y = generate_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)


Теперь пользуясь нашим простым критерием, попробуем классифицировать каких-то новых мышей

In [ ]:
X_test = np.random.uniform(14,30, 5)

def classify(X, threshold=21.5):
    y = np.zeros_like(X)
    y[X > threshold] = 1
    return y

total_len = 40
X,y = generate_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)
ax = plot_data(X_test, classify(X_test), total_len=total_len, s=300)

Но что если наши мыши, находятся тут?


In [ ]:
X_test = np.array([21.45, 22.5])

total_len = 40
X,y = generate_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)
ax = plot_data(X_test, classify(X_test), total_len=total_len, s=300)

С точки зрения нашего классификатора - все четко. Больше порогового значения - значит перевес, меньше - значит нормальные. Но с точки зрения здравого смысла, логичнее было бы классифицировать обоих мышей как нормальных, так как они значительно ближе к нормальным, чем к ожиревшим.

Вооружившись этим новым знанием, попробуем классифицировать наших отъевшихся мышек по умному. Возьмум крайние точки в каждом класстере. И в качестве порогового значения будем использовать среднее между ними

In [ ]:
X,y = generate_data(total_len=total_len)
most_obeese_normal = X[y==0].max()
most_normal_obeese = X[y==1].min()

threshold = np.mean([most_obeese_normal, most_normal_obeese])

X_test = np.array([21.5, 23])
ax = plot_data(X, y, total_len=total_len, threshold=threshold)
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

Мы можем посчитать насколько наша мышь близка к тому что бы оказаться в другом классе. Такое расстояние называется **margin**. И оно считается как $\mathrm{margin} = |\mathrm{threshold} - \mathrm{observation}|$

In [ ]:
margins = np.abs(X_test - threshold)
print(margins)

Соответственно, если мы посчитаем margins для наших крайних точек `most_obeese_normal` и `most_normal_obeese`, мы найдем самое большое возможное значение margin для нашего классификатора

In [ ]:
margin_0 = np.abs(most_obeese_normal - threshold)
margin_1 = np.abs(most_normal_obeese - threshold)
print(margin_0, margin_1)

Такой классификатор, мы называем **Maximum Margin Classifier**. Он хорошо работает в случае, когда все данные размечены аккуратно. Теперь рассмотрим более реалистичный пример, где что-то пошло не так

In [ ]:
def generate_realistic_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), np.random.uniform(24,33, total_len//2)])
    y = np.hstack([np.zeros(total_len//2), np.ones(total_len//2)])
    indx = np.where(X == X[y==1].min())[0]
    y[indx] = 0
    s = np.ones_like(X)*50
    s[indx] = 300
    return X,y,s

total_len = 40
X,y,s = generate_realistic_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len, s=s)

В таком случае, наш **Maximum Margin Classifier** работать не будет. Исходя из этого, мы можем придти к выводу, что наш классификатор очень чувствителен к выбросам. Давайте подумаем можно ли это как-то исправить?

Например, мы можем разрешить нашему классификатору ошибаться. Если мы будем использовать в качестве порогового значения не самое крайнее, а следующее за ним - мы промажем в классификации конкретно этой странной точки. Но в целом будем правы. Margin определенный таким образом называется **Soft margin**.

In [ ]:
total_len = 40
X,y,s = generate_realistic_data(total_len=total_len)

most_obeese_normal = np.sort(X[y==0])[-2]
most_normal_obeese = X[y==1].min()

threshold = np.mean([most_obeese_normal, most_normal_obeese])

X_test = np.array([20.5, 25])
ax = plot_data(X, y, total_len=total_len, threshold=threshold)
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

И тут то мы и подобрались к проблеме, из-за которой страдает практически все машинное обучение - **bias/variance trade-off**. Другими словами, перед тем, как мы позволили нашему классификатору ошибаться - у нас был маленький bias (предубеждение, смещение), и он плохо работал с новыми данными (high variance). После того как мы разрешили классификатору ошибаться - у классификатора теперь значительный bias, но при этом он лучше определяет класс - low variance

Но почему мы решили взять именно следующее значение? Почему не через 2? Откуда мы знаем что это лучший из возможных вариантов? А ни откуда не знаем. Что бы узнать какой из margins лучше, нам стоит численно это проверить и посчитать сколько раз мы ошибемся, если возьмем в качесве порогового значения между каждой парой точек. Для этого мы вновь воспользуемся *кросс-валидацией*.

In [ ]:
total_len = 40
X,y,s = generate_realistic_data(total_len=total_len)

indxs = []
accuracies = []
thresholds = []

X_test, y_test = generate_data(total_len=total_len)

for i in range(total_len//2):
    for j in range(total_len//2-1):
        most_obeese_normal = np.sort(X[y==0])[-i]
        most_normal_obeese = np.sort(X[y==1])[j]

        threshold = np.mean([most_obeese_normal, most_normal_obeese])
        
        y_pred = classify(X_test, threshold=threshold)
        accuracy = np.mean(y_test == y_pred)
        indxs.append((-i,j))
        accuracies.append(accuracy)
        thresholds.append(threshold)

print(f'Точность = {np.max(accuracies)*100}%')
print(f'Индексы, которые лучше всего подходят', indxs[np.argmax(accuracies)])

best_treshold = thresholds[np.argmax(accuracies)]
print('Лучшее пороговое значение %.2f'% best_treshold)

ax = plot_data(X_test, classify(X_test, threshold=best_treshold), 
               total_len=total_len, 
               s=200, 
               threshold=best_treshold)

Когда для классификатора используется **Soft Margin** - такой классификатор называют **Soft Margin Classifier** или по другому - **Support Vector Classifier**

### 2D класификация

Теперь рассмотрим пример, где мы измерили не только вес мышей, но и их длинну от хвоста до носа. Мы можем вновь применить наш метод Support Vector Classifier, и теперь классы разделяет не одно пороговое значение (по сути точка), а линия.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn import svm

def generate_2d_data(total_len=40):
    X, y = make_blobs(n_samples=total_len, centers=2, random_state=6)
    X[:,0] += 10 
    X[:,1] += 20 
    return X,y

def plot_data(X, y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Нормальные', 'С ожирением'])
    ax.set(xlabel='Масса, г', ylabel='Длинна, см');
    return ax

total_len = 40
X,y = generate_2d_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)

# Код для иллюстрации, разберемся что тут происходит позднее в лекции 
# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(X, y)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')

### 3D классификация

Если мы добивим еще одно измерение - возраст, мы обнаружим что наши данные стали трехмерными, а разделяет их теперь не линия, а плоскость

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def generate_3d_data(total_len=40):
    X, y = make_blobs(n_samples=total_len, centers=2, random_state=6, n_features=3)
    X[:,0] += 10 
    X[:,1] += 20 
    X[:,2] += 10
    return X,y

def plot_data(X, y, total_len=40, s=50, threshold=21.5):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs=X[:,0], ys=X[:,1], zs=X[:,2], c=y, s=s, cmap='Set1')
    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    ax.plot_surface(XX, YY, XX*YY*0.2, alpha=0.2)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Нормальные', 'С ожирением'])
    ax.set(xlabel='Масса, г', ylabel='Длинна, см', zlabel='Возраст, дни');
    return ax

total_len = 40
X,y = generate_3d_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)


Соответственно, если бы у нас было 4 измерения и больше (например вес, длинна, возраст, кровяное давление), то многомерная плоскость которая бы разделяла наши классы - называлась бы **гиперплоскость** (рисовать мы ее конечно же не будем). Чисто технически, и точка, и линия - тоже гиперплоскости. Но все же гиперплоскостью принято называть то, что нельзя нарисовать на бумаге.

### SVM

Но данные не всегда разделяются так хорошо как в случае нашего мышиного датасета. Напрмиер рассмотрим такой пример: у нас есть данные по дозировке лекарства и 2 класса - пациенты которые поправились, и те, скажем так что бы никого не расстраивать, которым лучше не стало

In [ ]:
def generate_patients_data(total_len=40):
    X = np.random.uniform(0,50, total_len)
    y = np.zeros_like(X)
    y[(X > 15) & (X < 35)] = 1
    return X,y

def plot_data(X, y, total_len=40, s=50):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=y, s=s)

    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Поправились', 'Болеют'])
    ax.set(xlabel='Доза, мг');
    return ax

total_len = 40
X,y = generate_patients_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)

Соответственно мы не можем найти такое пороговое значение, которое будет разделять наши классы на больных и здоровых, а следовательно и Support Vector Classifier работать тоже не будет. И вот тут-то мы наконец-то добрались до **Support Vector Machines**. Для начала, давайте преобразуем наши данные таким образом, что бы они стали 2х мерными. В качестве значений по оси Y будем использовать дозу возведенную в квадрат (**доза**$^2$).

In [ ]:
def plot_data(X, y, total_len=40, s=50):
    ax = sns.scatterplot(x=X[0,:], y=X[1,:], hue=y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Поправились', 'Болеют'])
    ax.set(xlabel='Доза, мг');
    ax.set(ylabel='Доза$^2$');
    return ax

total_len = 40
X_1,y = generate_patients_data(total_len=total_len)
X2 = X_1**2
X = np.vstack([X_1, X2])

plot_data(X, y, total_len=40, s=50)

Теперь мы можем вновь использовать Support Vector Classifier для классификации

In [ ]:
plot_data(X, y, total_len=40, s=50)

x = np.linspace(0,50,50)
xs = [X[0,:][y==1].min(), X[0,:][y==1].max()]
ys = [X[1,:][y==1].min(), X[1,:][y==1].max()]

# Calculate the coefficients. This line answers the initial question. 
coefficients = np.polyfit(xs, ys, 1)

# Let's compute the values of the line...
polynomial = np.poly1d(coefficients)
y_axis = polynomial(x)

# ...and plot the points and the line
plt.plot(x, y_axis, 'r--')

Внимательный слушатель, возможно уже догадался к чему все идет. Основная идея Support Vector Machine такая:
* Начинаем в низкоразмерном пространстве (в случае с дозами - одномерном)
* Превращаем наши данные в многомерные
* Применяем Support Vector Classifier


Но тут возникает резонный вопрос - почему мы решили возвести в квадрат? Почему не в куб? Или наоборт не извлечь корень? Как нам решить какое преобразование использовать?

SVM определяет способ, с помощью которого будет преобразовывать данные с помощью некой функции, которая называется **Kernel Function**. Kernel Function может, например быть полиномом (**Polynomial Kernel Function**), который имеет параметр $d$ - сколько размерностей выбрать. 
Для тех кто забыл, полином от степени n записывается в виде:

$a_dx^d + a_{d-1}x^{d-1} + a_{d-2}x^{d-2} + ... + a_0$

где $a_d$ - какое-то натуральное число.

Например полином степени d=1 можно записать в виде:

$a_1x^1 + a_0$

Полином степени d=2:

$a_2x^2 + a_1x^1 + a_0$

И так далее.

SVM перебирает значения $d$ по очереди в заданых пользователем пределах (например от 1ого до 3х) и с помощью кросс-валидации считает **какая ошибка классификатора при заданном $d$**. То $d$ при котором ошибка меньше всего фиксируется и используется для классификации новых точек.

**Polynomial Kernel Function** можно переписать в следующем виде: $(a \times b + r)^d$, где $a$ и $b$ точки (вектора), которые мы сравниваем, $d$ - степерь полинома, а $c$ - свободный параметр, позволяющий компенсировать влияние членов высшего порядка на члены низшего порядка в полиноме. Когда c = 0, ядро называется однородным.

Давайте рассмотрим конкретный пример, возьмем наши дозировки лекарств. И посчитаем однородную Polynomial Kernel Function для $d$ от 1 до 3.

In [ ]:
def generate_patients_data(total_len=40):
    X = np.random.uniform(0,50, total_len)
    y = np.zeros_like(X)
    y[(X > 15) & (X < 35)] = 1
    return X,y

def plot_data(X, y, total_len=40, s=50):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=y, s=s)

    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Поправились', 'Болеют'])
    ax.set(xlabel='Доза, мг');
    return ax

total_len = 40
X,y = generate_patients_data(total_len=total_len)
ax = plot_data(X, y, total_len=total_len)

In [ ]:
X.shape

Еще часто используют **Radial Kernel Function**, или по другому ее еще называют **Radial Basis Function Kernel (RBF)** 

### Регрессия

#### Пример простой линейной регрессии

Теперь ненадолго отвлечемся от SVM и рассмотрим другую задачу. В этой задаче мы будем прогнозировать успеваемость студента, в зависимости от количества часов, которые он учил материал. Это простая задача линейной регрессии, поскольку она включает всего две переменные.

Регрессия - это статистический метод, используемый в финансах, инвестировании и других дисциплинах, который пытается определить силу и характер связи между одной зависимой переменной (обычно обозначаемой Y) и рядом других переменных (известных как независимые переменные). В задачах регрессии, как и раньше, мы будем пытаться минимизировать ошибку между предсказанием и истинными данными. 

"Регрессия" происходит от слова "регресс", которое, в свою очередь, происходит от латинского "regressus" - возвращаться (к чему-либо). В этом смысле регрессия - это техника, которая позволяет "вернуться назад" от беспорядочных, трудно интерпретируемых данных к более четкой и осмысленной модели.

Загрузим датасет

In [ ]:
from IPython.display import clear_output 
!wget http://edunet.kea.su/repo/src/L02_Linear_classifier/datasets/student_scores.csv
clear_output()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Посмотрим что там в нем. Видим что у нас есть два признака - часы и результаты

In [ ]:
dataset = pd.read_csv("/content/student_scores.csv")
print(dataset.shape)
dataset.head()

Построем график зависимости одного от другого, а так же отобразим распрпеделения каждой из переменных

In [ ]:
import seaborn as sns
sns.jointplot(data=dataset, x="Scores", y="Hours")

Разделим наши данные на train и test

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1].values # значения столбца Hours
y = dataset.iloc[:, 1].values # значения столбца Score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Теперь создадим модель для линейной регрессии. Что бы не писать с нуля, воспользуемся готовой из библиотееки `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() # создаем модель линейной регрессии

И обучим ее

In [ ]:
regressor.fit(X_train, y_train) # обучаем на наших данных

Посмотрим что получилось

In [ ]:
X_train.shape

In [ ]:
x_points = np.linspace(
    min(X_train), max(X_train), 100
)  # создаем 100 точек в пределах min/max X_train
y_pred = regressor.predict(x_points)  # предсказываем результат для этих точек

plt.figure(figsize=(8, 5))
plt.plot(X_train, y_train, "o", label="Scores")
plt.plot(x_points, y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки

In [ ]:
y_pred = regressor.predict(X_test)  # делаем предсказание тестовой выборки

x_points = np.linspace(
    min(X_test), max(X_test), 100
)  # создаем 100 точек в пределах min/max X_train
y_pred = regressor.predict(x_points)  # предсказываем результат для этих точек

plt.figure(figsize=(8, 5))
plt.plot(X_test, y_test, "o", label="Scores")
plt.plot(x_points, y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Выглядит не плохо

Посчитаем метрики для наших значений

In [ ]:
from sklearn import metrics

y_pred = regressor.predict(X_test)

print("Mean Absolute Error: %9.2f" % metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error: %10.2f" % metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: %5.2f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

#### Геометрическая интерпретация 

Теперь, когда мы разобрались с тем что такое регрессия и с чем ее едят, вернемся к нашим картинкам. Как можно применить регрессию для классификации?

Предположим у нас есть только 2 класса. Как можно использовать регрессию для того что бы определить относится ли изображение к классу 0 или к классу 1? В упрощенном варианте, задача будет состоять в том, что бы провести разделяющую плоскость (прямую) между 2-мя классами. Например мы можем провести прямую через 0. 


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/2lecture_classifier_2.png" width="270">





Рассмотрим другую ситуацию, в этом случае, мы не можем просто провести прямую через 0. Но можем отступить от 0 на какое то расстояние и провести ее там. Вспомнил что уравнение прямой это $y=wx+b$, где $b$ - это смещение (*bias*). Соответственно если b != 0, то прямая через 0 проходить не будет, а будет проходить через значение b.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/2lecture_classifier.png" width="270">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)


Если у нас есть несколько классов (несколько шаблонов), мы можем для каждого из них посчитать уравнение $y_{i} = w_{i}x_{i}+b_{i}$.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-09.jpg" width="400">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)





На картинке нас интересуют 3 класса. Соответвенно мы можем записать систему линейных уравненией:

\begin{aligned}
y_{0} = w_{0}x_{0} + b_{0} \\
y_{1} = w_{1}x_{1} + b_{1} \\
y_{2} = w_{2}x_{2} + b_{2} \\
\end{aligned}

#### Добавление смещения

Мы их можем собрать в матрицу, тогда получится следующее:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-10.jpg" width="750">

У нас есть матрица коэффициентов, которые мы каким-то образом подобрали, пока ещё не понятно как. Есть вектор $x$, соответствующий изображению. 

Мы умножаем вектор на матрицу, получаем нашу гиперплоскость для четырехмерного пространства в данном случае. Чтобы оно не лежало в 0, мы должны добавить смещение. И мы можем сделать это после, но можно взять и этот вектор смещения (вектор **b**) просто приписать к матрице **W**.

Что будет выходом такой конструкции? Мы умножили матрицу весов на наш вектор, соответствующий изображению, получили некоторый отклик. По этому отклику мы так же, как и при реализации метода ближайшего соседа можем судить: если он больше остальных, то мы предполагаем, что это кошка.


In [ ]:
import numpy as np

img = np.array([56, 231, 24, 2])
w_cat = np.array([0.2, -0.5, 0.1, 2.0])
print("Изображение ", img)
print("Вектор весов ", w_cat)
print("Умножаем вектор изображения и весов ", img * w_cat)
print("Считаем сумму ", (img * w_cat).sum())
print("Добавляем смещение ", (img * w_cat).sum() + 1.1)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-07.jpg" width="600">


Собирая все вместе, получаем какое-то компактное представление, что у нас есть некоторая функция, на вход которой мы подаем изображение, и у нее есть параметры (веса). Пока происходит просто умножение вектора на матрицу, в дальнейшем это может быть что-то более сложное, функция будет представлять какую-то более сложную модель. А на выходе ( для классификатора) мы получаем числа, которые интерпретируют уверенность модели в том, что изображение принадлежит к определенному классу.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-11.jpg" width="450">



Соответственно, эти коэффициенты, которые являются весами модели, надо каким-то образом подбирать. Но прежде, чем подбирать коэффициенты, давайте определимся со следующим: как мы будем понимать, что модель работает хорошо или плохо? Ведь она возвращает достатотчно абстрактные числа, которые нужно уметь интерпретировать.


# Оценка результата

## Функция потерь SVM-loss



Вернемся к Support Vector Machine. Мы разобрали, как в принципе работает метод опорных векторов (SVM) — набор схожих алгоритмов обучения с учителем, использующихся для задач классификации и регрессионного анализа. Особым свойством метода опорных векторов является непрерывное уменьшение ошибки классификации и увеличение зазора. 

Теперь разберемся как SVM работает на практике. Вновь вернемся к нашему датасету CIFAR10. Мы же уже утверждали, что SVM работает с векторами (в частном случае с точками), а значит и с векторами размером (H,W,C) SVM работать тоже будет.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-12.jpg" width="550">

По аналогии с тем, что мы уже делали, мы можем сравнивать отклик на ключевой класс  (про который нам известно, что он на изображении, так как у нас есть метка этого класса) с остальными. Соответственно, мы подали изображение кошки и получили на выход вектор. Чем больше значение, тем больше вероятность того, что, по мнению модели, на изображении этот класс. Для кошки в данном случае это значение 2.9. Хорошо это или плохо? Нельзя сказать, пока мы не проанализировали остальную часть вектора. Если бы мы могли посмотреть на все значения в векторе, мы бы увидили, что есть значения больше, то есть в данном случае модель считает, что это собака, а не кошка, потому что для собаки значение максимально. 

На основании этого можно построить некоторую оценку. Давайте смотреть на разницу правильного класса с неправильными. В зависимости от того, насколько уверенность в кошке будет больше остальных, настолько будет наша модель хорошей.

Но поскольку нам важна не работа модели на конкретном изображении, а важно оценить ее работу в целом, то эту операцию нужно проделать либо для всего датасета, либо для некоторой выборки, которую мы подаем на вход и подсчитываем средний показатель. Этот показатель (насколько хорошо работает модель), называется функцией потерь, или **loss функцией**. Называется она так потому, что она показывает не то, насколько хорошо работает модель, а то, насколько плохо. 

Дальше будет понятно, почему так удобнее (разница только в знаке). Как это посчитать для всего датасета?

Мы каким-то образом считаем loss для конкретного изображения, потом усредняем по всем изображениям.

Дано: 3 учебных примера, 3 класса. При некотором W баллы f (a, W) = Wx равны:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-13_1.jpg" width="450">

Функция потерь показывает, насколько хорош наш текущий классификатор.

Дан датасет примеров:

$\begin{Bmatrix} (x_i,y_i)  \end{Bmatrix}_{i=1}^N 	$

Где **$x_i$** изображение и **$y_i$** метка (число).

Потери по набору данных — это среднее значение потерь для примеров:

$ L = {1 \over N}\sum_iL_i(f(x_i,W),y_i)$


Снова возьмём наши обучающие примеры **$x_i$** и метки классов **$y_i$**. Представим прогнозы классификатора (оценки) в виде векторов: **$s = f(x_i, W)$**. Для каждого отдельного примера просуммируем значения оценок всех **$y$**, кроме истинной метки **$y_i$**. То есть, мы получим сумму значений всех неправильных категорий. 

Теперь посчитаем разницу между ложными прогнозами и истинной оценкой: **$s_j − s_yi$** ; **$j≠y_i$**. Если истинная оценка больше, чем сумма всех неправильных прогнозов плюс некоторый дополнительный «зазор» (установим его равным 1) — значит, полученный балл для правильной категории намного превосходит любую ошибочную оценку. Это и будет наша функция потерь.

В символьном виде это выглядит так:


$f(n) = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$





## Вычисление SVM loss


 Не самая мощная, но достаточно интуитивно понятная loss функция –  SVM loss. 
 
Логика такая: если у нас уверенность модели в правильном классе большая, то модель работает хорошо и loss для данного конкретного примера должен быть равен нулю. Если есть класс, в котором модель уверена больше, чем в правильном, то loss должен быть не равен нулю, а отображать какую-то разницу, поскольку модель сильно ошиблась. При этом есть ещё одно соображение: что будет, если на выходе у правильного и ошибочного класса будут примерно равные веса? То есть, например, у кошки было бы 3.2, а у машины не 5.2, а 3.1. В этом случае ошибки нет, но понятно, что при небольшом изменении в данных (просто шум) скорее всего она появится.
 


То есть модель плохо отличает эти классы. Поэтому мы и вводили некоторый зазор, который должен быть между правильным и неправильным ответом. 

Посмотрим на изображение снизу, у нас есть два класса, фиолетовые треугольники и синие квадраты разделенные зазором. Также можем увидеть желтые треугольники и квадраты, это ошибочно распознанные классы.

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/svm_decision_boundary.png" width="300">


И тоже учитывать его в loss функции: то есть сравнивать его результат для правильного класса не с чистым выходом для другого, а добавить к нему некоторую дельту (в данном случае – 1(единица)). Смотрим: если разница больше 0, то модель работает хорошо и loss равно нулю. Если нет, то мы возвращаем эту разницу и loss будет складываться из этих индивидуальных разниц.

$f(n) = \sum_{j\neq y_i}\begin{cases}
  0,  & \mbox{если } s_{y_i}\geq s_j+1\mbox{} \\
  s_j-s_{y_i}+1, & \mbox{если наоборот, то} \mbox{}
\end{cases}$

$=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Ниже пример того, как считается loss.


Считаем функцию потерь для 1го изображения:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM1.jpg" width="650">

Также считаем потери для 2го и 3его изображения:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM2_1.jpg" >

Значения losses получились следующие:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-SVM4.png" width="450">

Считаем среднее значение loss для всего датасета:

$ L = {1 \over N}\sum_{i=1}^N L_i$

$L={2.9 + 0 + 12.9 \over 3} = 5.27$


SVM loss

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$


Как записать это в коде:

In [ ]:
import pickle
import numpy as np
def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


x_train = np.zeros((0, 3072))
y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    x_train = np.append(x_train, np.array(raw[b"data"]), axis=0)
    y_train = np.append(y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b"data"])
y_test = np.array(test[b"labels"])

labels_ru = [
    "Самолет",
    "Автомобиль",
    "Птица",
    "Кошка",
    "Олень",
    "Собака",
    "Лягушка",
    "Лошадь",
    "Корабль",
    "Грузовик",
]

print(x_train.shape)
print(x_test.shape)

In [ ]:
W = np.random.randn(10, 3072) * 0.0001 # создаем случайные веса

In [ ]:
assa = np.maximum(0, W.dot(x_train[0]) - W.dot(x_train[0])[6] + 1)
assa[6] = 0
np.sum(assa)

In [ ]:
W = np.random.randn(10, 3072) * 0.0001  # создаем случайные веса


def Li_svm(x, y, W):
    y = int(y)  # меняем тип данных, чтобы использовать как индекс
    scores = W.dot(x)  # умножаем веса на изображение
    margins = np.maximum(0, scores - scores[y] + 1)  # считаем loss
    margins[y] = 0  # не учитываем текущий классс равнвый 1
    return np.sum(margins)  # возвращаем сумму значений


L0 = Li_svm(x_train[0], y_train[0], W)
print("Loss on first image", L0)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-15.jpg" width="900">

Собираем все вместе: считаем loss для всего датасета, усредняя ее.


Как будет выглядеть график этой SVM-loss?

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/hinge_loss.gif" width="400">


Функция потерь для ситуации, когда зазор большой, будет равна 0. Она будет меняться только тогда, когда у нас есть ошибка. Причем будет меняться линейно.

Как мы можем использовать эту loss функцию? Допустим, мы знаем, что у нас модель будет работать не очень хорошо (выдает большую ошибку). Можно попытаться найти минимум этой loss функции, то есть подобрать такое W, когда loss функция обратится в 0  (в данном случае), а в общем случае (если какое-то количество данных нельзя обратить в 0), мы можем попытаться найти ее минимум.



# Обновления весов методом градиентного спуска

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-18.png" width="750">

Идея в следующем: у нас есть некоторая поверхность, на которой будет точка минимума функции. Мы должны обновлять веса таким образом, чтобы смещаться в сторону этой точки. Loss показывает, как возрастает функция потерь. Так как потери - это плохо, мы должны их минимизировать, то есть мы должны уменьшать веса в сторону, обратную росту этой функции.

Если переходить на случай n-мерного (в данном случае трехмерного) пространства, здесь достаточно очевидна аналогия с поверхностью: у нас есть поверхность земли, которая описывается координатами $x, y, z$. Если мы движемся по этой поверхности, высота $(z)$ будет зависеть от $x, y$. 

$x, y$ - это координаты, мы можем записать их как вектор. Наша функция будет работать с вектором координат и будет выдавать скаляр, третью координату. Если в качестве координат мы будем использовать веса нашей модели, а в качестве $z$ - loss, то аналогия станет полной. Наша задача сведется к тому, чтобы найти такой набор весов, при котором значение функции будет минимально. То есть мы окажемся в каком-то минимуме, где ошибка будет минимально возможна для этих данных.

Для того, чтобы предположить, где он может находиться, надо понимать, в какую сторону функция растет, а в какую - убывает. Для этого существует производная.

Поскольку у нас функция от нескольких переменных, если мы будем брать от нее производную, у нас получится вектор частных производных, то есть градиент этой функции, который будет показывать, как она меняется в каждом направлении.

## Градиент loss функции


Градиент — вектор, своим направлением указывающий направление наибольшего возрастания некоторой величины **φ**, значение которой меняется от одной точки пространства к другой (скалярного поля), а по величине (модулю) равный скорости роста этой величины в этом направлении.

Например, если взять в качестве **φ**  высоту поверхности земли над уровнем моря, то её градиент в каждой точке поверхности будет показывать «направление самого крутого подъёма», и своей величиной характеризовать крутизну склона.

Другими словами, градиент — это производная по пространству, но в отличие от производной по одномерному времени, градиент является не скаляром, а векторной величиной.

Для случая трёхмерного пространства градиентом скалярной функции **φ=φ(x,y,z)** координат **(x,y,z)** называется векторная функция с компонентами
$$\frac{\partial\varphi}{\partial x}, \frac{\partial\varphi}{\partial y}, \frac{\partial\varphi}{\partial z}$$.

Если **φ**  — функция **n** переменных **X1...xn**, то её градиентом называется **n**-мерный вектор
$$\frac{\partial\varphi}{\partial x_1},...,\frac{\partial\varphi}{\partial x_n}$$



компоненты которого равны частным производным **φ** по всем её аргументам.

Размерность вектора градиента определяется, таким образом, размерностью пространства (или многообразия), на котором задано скалярное поле, о градиенте которого идёт речь.
Оператором градиента называется оператор, действие которого на скалярную функцию (поле) даёт её градиент. Этот оператор иногда коротко называют просто «градиентом».


$W$ - матрица(вектор) весов

$L$ - функция потерь

$\partial W = W_2 - W_1$

$\partial L = L_2 - L_1$


$\frac{\partial L}{\partial W}=\begin{bmatrix}
\frac{\partial L}{\partial W_1} \\
\frac{\partial L}{\partial W_2} \\
... \\
\frac{\partial L}{\partial W_n}
\end{bmatrix}$


Наша задача будет сводиться к тому, что мы будем искать градиент loss функции по весам, которые будут состоять из производной по каждому направлению. Поскольку у нас здесь числа, можно считать этот градиент численно.

## Численный расчет производной

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-gradient_spusk.png" width="650">



Посчитаем градиент приближенно, воспользовавшись определением (в формуле аргумент обозначен как $x$, у нас же аргументом будет $W$):
На нулевом шаге у нас есть $W_0$ найдем $L_0 = Loss(f(W_0,x))$
Прибавим к первому элементу $W_0$ небольшую величину  $h$ = 0.0001 и получим новую матрицу весов $W_1$ отличающуюся от $W_0$ на один единственный элемент.

Найдем Loss от $\frac {W_1}  {L_1} = Loss(f(W_1,x))$
По определению производной $\frac {dL}{W_0} = \frac{( L_1 - L_0 )}   {h}$
Повторяя этот процесс для каждого элемента из $W$ найдем вектор частных производных то есть градиент $\frac{dL}{dW}$.

Плюсы:
Это просто. 

Проблемы:

1. Это очень долго, нам придется заново искать значение loss функции для каждого $W_i$.

2. Это неточно, так как по определению приращение $h$ бесконечно мало, а мы используем конкретное пусть и небольшое число. И если мы сделаем его слишком маленьким то столкнемся с ошибками связанными с округлением в памяти компьютера.


Поэтому данный метод может использоваться как проверочный. 
А на практике вместо него используется **аналитический расчет градиента**.


## Аналитический расчет производной от SVM Loss

### Простые производные 

$$x' = \frac {\delta x} {\delta x} = 1$$ 

$$(x^2)' = \frac {\delta x^2} {\delta x} = 2x$$

$$(\log x)'  = \frac {\delta \log x} {\delta x} = \frac 1 x $$

$$(e^x)'  = \frac {\delta e^x} {\delta x} = e^x $$

$$\frac {\delta cf(x)} {\delta x}= c \cdot \frac {\delta f(x)} {\delta x}$$

$$\frac {\delta f(x) + c} {\delta x}= \frac {\delta f(x)} {\delta x}$$

$c$ - константа, не зависящая от $x$

$$ \frac {\delta [f(x) + g(x)]} {\delta x} = \frac {\delta f(x)} {\delta x}  + \frac {\delta g(x)} {\delta x} $$ 


$$\frac {\delta (x^2 + y^3)} {\delta x} = 2x  $$
так как $y$ по отношению к $x$ - константа и мы меняем только $x$

$$\frac {\delta (x^2 + y^3)} {\delta y} = 3y^2  $$
так как $x$ по отношению к $y$ - константа и мы меняем только $y$

$$(e^y)'  = \frac {\delta e^y} {\delta x} = 0 $$

### Chain-rule

Производная функции $f(g)$:

$$\frac {\delta f} {\delta g}$$

Пусть $g$  на самом деле не просто переменная, а зависит от $h$. Тогда производная от $f$ по $g$ **не меняется**, а производная $f$ по $h$ запишется следующим образом:


$$\frac {\delta f(g(h))} {\delta h} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h}$$

Пусть теперь $h$ зависит от $x$. Все аналогично


$$\frac {\delta f(g(h(x)))} {\delta x} = \frac {\delta f} {\delta g} \frac {\delta g} {\delta h} \frac {\delta h} {\delta x}$$

Так можно делать до бесконечности, находя производную сколь угодно сложной функции. И, что важно - мы можем считать градиенты частями - посчитать сначала $f$ по $g$, потом $g$ по $h$....

$$\frac {\delta log(x^2 + 5)} {\delta x}$$

$$h = x^2 + 5$$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac {\delta log(h)} {\delta h} \frac {\delta h} {\delta x}$$ 

$$\frac {\delta log(h)} {\delta h} = \frac 1 h$$

$$\frac  {\delta h} {\delta x} = 2x $$

$$\frac {\delta log(x^2 + 5)} {\delta x} = \frac 1 {x^2 + 5} \cdot 2x = \frac {2x} {x^2 + 5}$$


###  Часть MSE-loss

$$loss = (y - \hat{y})^2 $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta (y - \hat{y})^2 } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = 2(y-\hat{y}) \cdot -1 = 2 (\hat{y} - y)$$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta w} = 2 x \cdot (\hat{y} - y) $$

### MSE-loss
$$MSE = \frac 1 N \sum_i(y_i - \hat{y_i})^2 $$

$y_i$ - константы
$\hat{y_i}$ - не являются функциями друг от друга 
$$\hat{y} = wx_i + b $$

$$\frac {\delta MSE} {\delta w} = \frac 1 N \sum \frac {\delta (y_i - \hat{y_i}) ^2} {\delta \hat{y_i}} \frac {\delta \hat{y_i}} {\delta w}$$




### Часть MAE-Loss

$$loss = |y - \hat{y}| $$

$$\hat{y} = wx + b $$

$$ \frac {\delta loss} {\delta w} = \frac {\delta loss} {\delta \hat{y}} \cdot \frac {\delta \hat{y}} {\delta w} $$

$$ \frac {\delta \hat{y}} {\delta w} = \frac {\delta wx + b} {\delta w} = x$$

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}|) } {\delta {(y - \hat{y})}}$$

Строго говоря, у модуля не существует производной в 0. 







In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

x = [i for i in range(-5, 6)]
y = [abs(i) for i in range(-5, 6)]

plt.figure(figsize=(8, 5))
plt.plot(x, y, label="y = |x|")
plt.title("y = |x|", size=20)
plt.legend()
plt.show()

Но мы можем сказать, что в этой точке производная равна 0.
Если аргумент модуля меньше 0, то производная будет -1.

Если больше +1

$$ \frac {\delta loss} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \frac {\delta y - \hat{y}} {\delta \hat{y}} = \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} \cdot -1 = - \frac {\delta |y - \hat{y}| } {\delta {(y - \hat{y})}} = - sign(y - \hat{y}) =  sign(\hat{y} - y)$$

<br>
<br>


In [ ]:
x = [i for i in range(-5, 1, 1)]
y = [i * 0 - 1 for i in range(6)]
x_1 = [i for i in range(0, 6)]
y_1 = [i * 0 + 1 for i in range(0, 6)]

plt.figure(figsize=(8, 5))
plt.plot(x, y, "b")
plt.plot(x_1, y_1, "b")
plt.plot(0, 0, "ro")
plt.plot(0, 1, "bo")
plt.plot(0, -1, "bo")
plt.show()

### Max-Loss

$$b = max(x, y)$$

$$b = x~~if~~x > y~~else~~y$$


$$\frac {\delta b} {\delta x} =  \frac {\delta x} {\delta x}~~if~~x > y~~else~~\frac {\delta y} {\delta x} = 1~~if~~x > y~~else~~0$$

Если $x > y$, то он оказал влияние на $b$. Иначе, его вклада в $b$ НЕ БЫЛО - градиент равен 0

 


### SVM-Loss

Из: 

$L_i=\sum_{j\neq y_i}max(0,s_j-s_{y_i}+1)$

Получаем:

$ \nabla_W	L(W) = {1 \over N}\sum_{i=1}^N \nabla_W L_i(x_i, y_i, W)$

##### $\color{brown}{\text{Дополнительная информация}}$

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g1.png" width="750"> 


Распишем формулу для $L_i$ через скалярное произведение.
Затем раскроем сумму.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g2.png" width="750"> 

Считаем производную по первому элементу матрицы $W$. 
Если у нас правильный класс - не первый элемент, то он больше нигде не встретится, кроме как в первой сумме. Поэтому все остальные суммы при подсчете этой частной производной будут константами. Поэтому этот градиент будет равен либо $x_i$,  либо 0 (если эта сумма меньше 0). 

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g3.png" width="750"> 

Аналогично для  $w_i$ частные производные будут равны $x_i$ либо нулю, когда выражение больше меньше 0, 
кроме случая: $j == yi$.  



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g4.png" width="750"> 


Эта ситуация для “правильного изображения”.



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_an_r_g5.png" width="750">  

Вот финальный результат.
Общие потери считаются по пакету изображений.
“1” в формулах  — это “if”.


### *Не обязательное задание:

Поставлю дополнительно 20 баллов тому, кто графически оформит результаты этого эксперимента и напишет вывод о зависимости качества обучения от learning_rate и batch_size (возможно, нужно будет добавить варианты их сочетаний и/или увеличить количество эпох до 20).

In [ ]:
from IPython.display import clear_output 
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz
!ls -l
clear_output()

In [ ]:
import numpy as np
import pickle

def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


x_train = np.zeros((0, 3072))
y_train = np.array([])
for i in range(1, 6):
    raw = unpickle(f"/content/cifar-10-batches-py/data_batch_{i}")
    x_train = np.append(x_train, np.array(raw[b"data"]), axis=0)
    y_train = np.append(y_train, np.array(raw[b"labels"]), axis=0)

test = unpickle("/content/cifar-10-batches-py/test_batch")
x_test = np.array(test[b"data"])
y_test = np.array(test[b"labels"])

labels_ru = [
    "Самолет",
    "Автомобиль",
    "Птица",
    "Кошка",
    "Олень",
    "Собака",
    "Лягушка",
    "Лошадь",
    "Корабль",
    "Грузовик",
]

print(f"Размерность обучающей выборки X: {x_train.shape}, y: {y_train.shape}")
print(f"Размерность тестовой выборки X: {x_test.shape}, y: {y_test.shape}")

In [ ]:
import random


class LinearClassifier:
    def __init__(self, labels, batch_size, random_state=42):
        self.labels = labels  # названия классов
        self.classes_num = len(labels)  # количество классов

        np.random.seed(
            random_state
        )  # устанаваливаем randim.seed чтобы можно было повторить случайные значения
        self.W = (
            np.random.randn(3073, self.classes_num) * 0.0001
        )  # создаем случайные веса, меняем размерность для добавления bias
        self.batch_size = batch_size  # значение batch_size

    def fit(self, x_train, y_train, learning_rate=1e-8):
        loss = 0.0  # обнуляем loss
        train_len = x_train.shape[0]  # сколько примеров
        indexes = list(range(train_len))  # список индексов для train_len
        random.shuffle(indexes)  # перемешиваем порядок индексов

        for i in range(
            0, train_len, self.batch_size
        ):  # для все выборки, с шагом batch_size
            idx = indexes[
                i : i + self.batch_size
            ]  # собираем индексы изображений для кажддого батча
            x_batch = x_train[idx]  # батч для X
            y_batch = y_train[idx]  # батч для Y

            x_batch = np.hstack(
                [x_batch, np.ones((x_batch.shape[0], 1))]
            )  # добавляем bias

            loss_val, grad = self.loss(x_batch, y_batch)  # считаем loss и градиент
            self.W -= learning_rate * grad  # обновляем веса

            loss += loss_val  # суммируем loss
        return loss / (train_len)  # высчитываем среднее значение loss

    def loss(self, x, y):
        current_batch_size = x.shape[0]  # текущий batch_size
        loss = 0.0  # обнуляем loss
        dW = np.zeros(self.W.shape)  # массив нулей для градиента
        for i in range(current_batch_size):  # по каждому изображению в batch_size
            scores = x[i].dot(
                self.W
            )  # умножаем значения изображения на веса, получим вектор размером 10
            correct_class_score = scores[
                int(y[i])
            ]  # получем текущее значения для верного класса
            above_zero_loss_count = 0  # счетчик для loss > 0
            for j in range(self.classes_num):  # проходимся по всем классам
                if j == y[i]:  # определяем текущий класс
                    continue
                margin = scores[j] - correct_class_score + 1  # считаем значение loss
                if margin > 0:  # если больше 0
                    above_zero_loss_count += (
                        1  # считаем сколько раз loss был больше нуля
                    )
                    loss += margin  # суммируем loss
                    dW[:, j] += x[i]  # задаем значения градиента
            dW[:, int(y[i])] -= above_zero_loss_count * x[i]  # корректируем
        loss /= current_batch_size  # считаем loss по текущему batch_size
        dW /= current_batch_size  # считаем градиент по текущему  batch_size
        return loss, dW

    def forward(self, x):
        x = np.append(x, 1)  # добавляем единицу в конец массива для bias
        scores = x.dot(self.W)  # получаем значения оценки
        return np.argmax(scores)  # возвращаем индекс максимального значения

In [ ]:
import time

def validate(model, x_test, y_test, noprint=False):
    correct = 0  # здесь храним количество верных предсказаний
    for i, img in enumerate(x_test):  # проходимся по всем изображениям в выборке
        index = model.forward(img)  # определяем класс изображения
        correct += (
            1 if index == y_test[i] else 0
        )  # если класс верный то увеличиваем correct на 1
        if noprint is False:  # если стоит флаг False
            if i > 0 and i % 1000 == 0:  # каждые 1000 значений
                print(
                    "Accuracy {:.3f}".format(correct / i)
                )  # выводим значения точност, количество верных ответов/количество попыток
    return correct / len(y_test)  # возвращаем общее значение точности

In [ ]:
print("Исследуем зависимость качества обучения от скорости:")

for lr in [1e-2, 1e-8]:
    for bs in [256, 2048]:

        print("-" * 50, "\n", "learning_rate =", lr, "\tbatch_size =", bs)
        print()
        lc_model = LinearClassifier(labels_ru, batch_size=bs)

        best_accuracy = 0
        for epoch in range(10):
            loss = lc_model.fit(x_train, y_train, learning_rate=lr)
            accuracy = validate(lc_model, x_test, y_test, noprint=True)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
            print(f"Epoch {epoch} \tLoss: {loss}, \tAccuracy:{accuracy}")

        print()
        print(f"Best accuracy is {best_accuracy} in {best_epoch} epoch")

# Выбор шага обучения

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-change_weight.png" width="450"> 

<!-- [Визуализация](https://docs.google.com/file/d/0Byvt-AfX75o1ZWxMRkxrUFJ2ZUE/preview)
 -->



Шаг обучения - некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время чтобы тот же процесс не шел слишком медленно (как на графике слева)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-learning_rate.png" > 

 

Пока изменения loss функции достаточно большие, сам по себе градиент тоже большой. За счет этого можно двигаться быстро. Когда он будет уменьшаться, мы должны оказаться рядом с этим минимумом, и шаг, который мы выбрали, не должен мешать этому процессу. 

Бывают ситуации, когда шаг можно менять в самом процессе обучения, когда в начале обучения модели шаг большой, потом, по мере того, как она сходится, чтобы более четко найти минимум, шаг можно изменить вручную. Но изначально его нужно каким-то образом подобрать, и это зависит от данных и от самой модели. Это тоже гиперпараметр, связанный с обучением.

# Выбор размера батча



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier-gd.png" width="500"> 

Мы с самого начала говорили о выборках, некоторого количества примеров. В дальнейшим мы будем называть их батчами. Батч (англ. *batch*) это некоторое подмножество обучающей выборки фиксированного размера.

При этом было не очень понятно, чем они мотивированны. Точнее, мы мотивировали это тем, что у нас много данных и мы не сможем их обработать все, и это правда. Даже если мы сможем загрузить все данные в память, нам нужно будет загрузить их и использовать при рассчете в том числе градиента. Это ещё более затратно. 

Зачем батчи нужны при рассчете loss? Если мы посчитаем loss по одному изображению, скорее всего он будет очень специфичен, и это движение, которое произойдет, будет направлено в сторону минимума, потому что по этому конкретному изображению мы улучшим показатели,что не отражает обобщения всех данных, поэтому мы используем батчи.

Нас в первую очередь интересует более общее направление, которые будут работать на большинстве наших данных. Поэтому если мы будем оптимизировать модели, исходя из одного элемента данных, путь будет витиеватый, и процесс будет происходить достаточно долго. Если же данные не помещаются в память, то **можно использовать батч того размера, который у нас есть**. Можно загрузить в память и считать по батчам градиент. В этом случае спуск будет более плавным, чем по одному изображению. 

Также при использовании всего датасета тоже есть свои минусы. **Не всегда загрузка всего датасета приводит к увеличению точности**. 

Подробнее: 
* [Методы оптимизации нейронных сетей
](https://habr.com/ru/post/318970/)
* [Обучение нейронной сети](https://alxmamaev.medium.com/deep-learning-на-пальцах-часть-1-341cfe021ef9)


# Регуляризация

Наша модель может переобучаться - она может просто поднастроить веса по тренировочную выборку. Особенно это характерно для больших моделей, когда число весов приближается к числу параметров. 






<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_regularisation.jpg" width="700"> 


L2 Regularization = weights decay

Идея состоит в том, что мы можем наложить некоторое требование на сами веса. Дело в том, что можно получить один и тот же выход модели при разных весах (выход модели соответствует умножению весов на $x$), при разных $w$ выход может быть идентичен.

Эти параметры задают некоторую аппроксимацию нашей целевой функции, аппроксимировать функцию можно двумя способами:
1. Использовать все имеющиеся данные и провести ее строго через все точки, которые нам известны; 
2. Использовать более простую функцию: (в данном случае линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым общим закономерностям, которые у них есть. 

Характерной чертой переобучения является второй сценарий, и сопровождается он, как правило большими весами. Введение L2-регуляризации приводит к тому, что большие веса штрафуются и предпочтение отдается решениям, использующим малые значения весов. 

Модель может попробовать схитрить и по-другому - использовать все веса, все признаки, даже незначимые, но с маленькими коэффициентами. С этим L2-loss поможет хуже, так как он не сильное штрафует мелкие веса. Результатов его применения - малые значения весов, которые использует модель

В этом случае на помощь приходит L1-loss, который штрафует вес за сам факт отличия его от нуля. Но и штрафует он все веса одинаково. Результат его применения - малое число весов, которые использует модель в принципе. 



<img src="https://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/losses.gif" alt="alttext" style="width: 450px;"/>


Это лоссы можно комбинировать - получится Elastic Net




$\lambda=$ regularization strength (hyperparameter)

$L(W)=\underbrace{\frac1N\sum_{i=1}^NL_i(f(x_i,W),y_i)}_{\textbf{Data loss} }+\underbrace {\lambda R(W)}_{\textbf{Regularization}}$

Берем сумму всех весов по всей матрице $w$, и добавляем ее к loss. Соответственно, чем больше будет эта сумма, тем больше будет суммарный loss. 


В дальнейшем проблема с переобучением будет вставать довольно часто. Методов регуляризации модели существует достаточно много. Этот — один из базовых, который будет использоваться практически во всех оптимизаторах, с которыми познакомимся позже.

# Cross-entropy Loss

[Отличное видео от Statquest про энтропию](https://www.youtube.com/watch?v=YtebGVx-Fxw)



На Cross-entropy Loss построены практически все модели, про которые будет идти речь.

Это оценка результата и обновление весов с использованием градиента. У нас один слой, поэтому мы достаточно легко посчитали от него градиент. Если слоев будет много, сделать это будет сложней.

Рассмотрим ещё одну loss-функцию. 

SVM loss хороша тем, что интуитивно понятно, как ее посчитать. Но проблема состоит в том, что модель выдает некоторые числа, которые сами по себе невозможно интерпретировать. Сами по себе выходы модели мало что означают. Было бы неплохо, если бы мы сразу, глядя на выход модели для кошки, могли бы их как-то интерпретировать, не просматривая остальные веса. Хорошо бы, чтобы модель выдавала не какую-то абстрактную уверенность, а **вероятность** того, что по ее мнению, на картинке изображена кошка.


## Переход к вероятностям

**Softmax**

[Видео от StatQuest, которое объясняет Softmax](https://www.youtube.com/watch?v=KpKog-L9veg)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier_softmax.png" width="750">

Перейти к вероятностям мы сможем, проведя с весами некоторые не очень сложные математические преобразования. 

На слайде выше показано, почему выходы модели часто называют [logit’ами](https://en.wikipedia.org/wiki/Logit). Если предположить, что у нас есть некая вероятность, от которой мы берем такую функцию (logit), то она может принимать значения от нуля до бесконечности. Мы можем считать, что выходы модели - это logit’ы. 




Например мы могли бы просто взять индекс массива, в котором значение (logit) максимально. Предположим, что наша сеть выдала следующие значения:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
logits = [
          5.1, # кошка
          3.2, # машина
          -1.7, # лягушка
]

Тогда, что бы узнать какой класс наша сеть предсказала, мы могли бы просто взять `argmax` от наших `logits`

In [ ]:
print('Предсказанный класс = %i (кошка)' % (np.argmax(logits)))

Но от argmax нельзя посчитать градиент, так как производная от константы равна 0. Соответствеенно, если бы мы вставили производную от argmax в градиентный спуск, мы бы получили везде нули, и соответственно, наша сеть бы вообще ничему не научилась

In [ ]:
plt.scatter(np.arange(3), [1,0,0], color='red', s=50)

А мы бы хотели получить не logit’ы, а настоящую вероятность на выходе модели. Да еще и таким образом, что бы от наших вероятностей можно было бы посчитать градиент. Для этого мы можем применить к нашим логитам функцию **Softmax**

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier_softmax2.jpg" width="1300">

Мы можем провести над логитами операцию экспоненцирования, то есть число Эйлера (2.71828) **возвести в степень**, соответствующую этому выходу. В результате мы получим вектор, гарантировано неотрицательных чисел (потому что мы ввели неположительное число в степень, пускай даже отрицательную. То есть выходы могут быть маленькие, но всегда положительные). 



Дальше, чтобы можно было интерпретировать эти числа как вероятности, их сумма должна быть равно единице. Мы должны их нормализовать, то есть **поделить на сумму**. Это преобразование называется **Softmax функцией**.

**Получаются вероятности**, то есть числа, которые можно интерпретировать таким образом.

$\text{Softmax}_\text{кошка} = \frac{e^{5.1}}{e^{5.1} + e^{3.2} + e^{-1.7}}$

In [ ]:
def softmax(logits):
    return np.exp(logits)/np.sum(np.exp(logits))

print(softmax(logits))
print('Сумма = %.2f' % np.sum(softmax(logits)))

Можно обратить внимание, что Softmax, никоим образом не поменял порядок значений. Самому большому logit'у соотвеетствует самая большая вероятность, а самому маленькому, соответственно самая маленькая

Посмотрим на графиках. Возьмем массив случайных логитов и применим к ним softmax

In [ ]:
rand_logits = np.linspace(-1,1,50)
fig,ax = plt.subplots(ncols=2)

ax[0].plot(np.arange(50), rand_logits)
ax[0].set_title('Логиты')
ax[1].plot(np.arange(50), softmax(rand_logits))
ax[1].set_title('Softmax')

**Cross-entropy / log loss**

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_cross-entropy.png" width="800">


Теперь, с использованием такого вектора можно определить другую loss функцию. Если не вникать в детали, можно **взять от нее логарифм**. Тогда loss от такого выхода будет выглядеть вот таким образом:

$ L_i =  -log(\frac{e^{s_{y_i}}}{\sum_j e^{s_j}})$

как нормализованный выход от верного класса на сумму остальных, к которому добавили минус. 

Получится график loss (слева). Его плюс заключается в том, что у него нет участка с плато, практически по всей длине функция получается гладкой, с хорошими мощными производными. Когда loss большой, производная тоже большая, и за счет этого можно быстро обучать модель. Для кусочно-линейной функции потерь она же равна константе.

До тех пор, пока модель не будет работать с точностью 100%, то есть пока все остальные выходы не станут нулевыми, мы можем продолжить обучение, даже если у нас нет явной регуляризации.

Осталось выяснить, почему такая простая на вид функция **называется так сложно** - Кросс-энтропия. 



## Кросс-энтропия

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_cross-entropy2.png" >

Если мы рассматриваем выходы нашей модели как вероятности, то мы можем их сравнивать. У нас есть номер правильного класса. Соответственно, можно сказать, что вероятность этого класса - единица, а всех остальных 0, и получить таким образом вероятностное распределение. Выход модели тоже можно интерпретировать как вероятности (тоже можно получить вероятностное распределение). И для работы с этими распределением есть некоторый математический аппарат, который основан на понятии энтропии, который ввел Клод Шеннон.


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_sun.png" >


Идея в следующем: у нас есть некоторые данные, которые мы передаем по каналу связи. Например, у нас есть метеостанция, которая сообщает прогноз погоды. Допустим, она может передавать 8 вариантов прогноза. Мы в каждый момент времени получаем от нее сообщение. Предположим, что потребуется 3 бит, чтобы передавать это сообщение. 

Допустим, мы знаем некую вероятность, с которой в нашем регионе может наступить хорошая либо плохая погода. То есть мы знаем вероятностное распределение, по которыму у нас, допустим, солнечное место, где почти не бывает бури. Если мы знаем об этом, мы можем сэкономить на передаче информации. Мы можем закодировать наиболее вероятные сообщения двумя битами, причем придумать такие коды, чтобы они почти не пересекались. Они будут начинаться с нуля и передавать два бита вместо трех. Таким образом можно сэкономить на передачи информации. То есть идея такая: **если мы знаем, что события маловероятны, мы можем кодировать их более длинной цепочкой, а более вероятные события - более короткими цепочками**. В этом случае в среднем количество информации, которое можно передать, сократиться.

**Формула Шеннона позволяет посчитать, насколько сильно мы можем сэкономить для конкретного вероятностного распределения**. То есть если подставить эти данные в формулу, то получим 2,23.

На практике реализовать это, используя биты, возможно, не выйдет, но это свойство данного вероятностного распределения можно посчитать. Оно несёт в себе некоторое количество полезной информации, соответствующее этой энтропии.

**Что же такое кросс-энтропия?**

Давайте представим, что мы каким-то образом начали кодировать эту информацию, прошили погодную станцию так, что она может эту кодированную информацию передавать, а потом перенесли в другой регион, где ситуация противоположная. И мы кодируем события, которые стали частыми, длинными цепочками, а маловероятные - короткой, и у нас в этом случае явно передается больше информации, чем мы могли бы, если бы сделали все банально. 

Кросс-энтропия позволяет посчитать, насколько большая будет потеря в данном случае. То есть это некий **способ сравнить между собой вероятностные распределения**.

Что, собственно говоря, отображено в формуле, потому что мы считаем кросс-энтропию по нашему вектору $P$ и $Q$. Вектор $P$ состоит из нулей и единиц, соответственно, во всех случаях, кроме одного, это выражение будет нулевым, кроме одного случая с единицей. Случай с единицей соответствует нашему правильному классу. Поэтому сумма исчезает, остаются логарифм и вектор, а вероятность для правильного класса мы считаем как Soft max. Отсюда название кросс-энтропия.  


(В теории информации кросс-энтропия между двумя распределениями вероятностей $p$ и $q$  по одному и тому же базовому набору событий измеряет среднее число битов, необходимых для идентификации события , взятого из набора, если схема кодирования, используемая для набора, оптимизирована для оценочного распределения вероятностей $q$, а не для истинного распределения $p$.)

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img_license/L02_Linear_classifier_cross-entropy3.png" width="900">

Аналогичное обоснование: есть понятие дивергенция, которая позволяет оценить расхождение между вероятностными распределениями, которая нам здесь и нужна. Есть дивергенция **Кульбака-Лейблера**, которая выражается через кросс-энтропию, а энтропия от нашего вектора нулевая. Поэтому фактически здесь кросс-энтропия равна дивергенции, которая показывает, насколько непохожи два распределения. 


<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_softmax3.jpg" width="800">


Эта функция очень популярна, она используется при обучении реальных моделей на практике. Поскольку внутри находится Softmax, часто ее называют Softmax-классификатором, что, строго говоря, не совсем верно. Здесь функция потерь - кросс-энтропия, Softmax же просто используется внутри нее.


## Градиент Cross Entropy Loss




[Cross Entropy Loss](http://machinelearningmechanic.com/deep_learning/2019/09/04/cross-entropy-loss-derivative.html)

Подавляющее большинство методов классификации и регрессии сформулированы в терминах евклидовых или метрических пространств, то есть подразумевают представление данных в виде векторов одинаковой размерности. В реальности мы имеем дело с категориальными данными, которые нужно представить в виде вектора. One Hot Encoding подразумевает создание вектора, размером равным количеству классов, все из которых равны нулю за исключением одного. На позицию, соответствующую значению класса мы помещаем 1.

Например, если у нас 10 классов, тогда для каждого класса соотвентственно 

Класс 0 = [**1**,0,0,0,0,0,0,0,0,0]\
Класс 1 = [0,**1**,0,0,0,0,0,0,0,0]\
...\
Класс 9 = [0,0,0,0,0,0,0,0,0,**1**]


In [ ]:
n_classes = 10
y_class = 5

one_hot = np.zeros(n_classes)
one_hot[y_class] = 1
yi = one_hot
print(yi)

##### $\color{brown}{\text{Дополнительная информация}}$

$$ -L = \sum_i y_i \log p_i = \sum_i y_i \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

$$s_{y_i} = w_i x$$

$$ \dfrac {\delta L} {\delta w_i} = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} $$

$$\dfrac {\delta s_{y_i}} {\delta w_i} = x$$

Только один $y_k = 1$


$$ -L = y_k \log p_i = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$

i = k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_i}} - \log  \sum_j e^{s_{y_j}}  = s_{y_i} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} = 1 - \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = 1 - \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = 1 - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i - 1 $$

$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = (p_i - 1) x $$

i != k

$$ -L = \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}}) = \log e^{s_{y_k}} - \log  \sum_j e^{s_{y_j}}  = s_{y_k} - \log  \sum_j e^{s_{y_j}}$$

$$\dfrac {\delta -L} {\delta s_{y_i}} = - \dfrac 1 {\sum_j e^{s_{y_j}}} \cdot \dfrac {\delta {\sum_j e^{s_{y_j}}}} {\delta s_{y_i}} =  \dfrac 1 {\sum_j e^{s_{y_i}}} \cdot \dfrac {\delta e^{s_{y_i}}} {\delta s_{y_i}} = \dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}} = - p_i$$

$$\dfrac {\delta L} {\delta s_{y_j}} = p_i $$
$$ \dfrac {\delta L_i} {\delta w_i}  = \dfrac {\delta L} {\delta s_{y_i}} \dfrac {\delta s_{y_i}} {\delta w_i} = p_i x $$



## Практическое вычисление SoftMax

Когда мы возводим число в степень, может получиться очень большое число и произойти переполнение памяти. Неизвестно, какие будут у модели выходы. 

In [ ]:
f = np.array([123, 456, 789])
p = np.exp(f) / np.sum(np.exp(f))
print(p)

Поэтому мы можем вычесть из каждого $s_i$ пооложительную константу, чтобы уменьшить значения экспонент. В качестве константы можно выбрать максимальный элемент этого вектора, тогда у нас гарантированно не будет очень больших чисел и такой способ будет работать более стабильно.

$$M = \max_j s_{y_{j}}$$
$$s^{new}_{y_{i}}  = s_{y_{i}} - M $$

$$ \dfrac {e^{s^{new}_{y_{i}}}} {\sum_j e^{s^{new}_{y_{j}}}}  = \dfrac {e^{s_{y_{i}} - M }} {\sum_j e^{s_{y_{j}} - M }} = \dfrac {e^{s_{y_{i}}}e ^ {-M}} {\sum_j e^{s_{y_{j}}} e ^ {-M}} = \dfrac {e ^ {-M} e^{s_{y_{i}}}} {e ^ {-M} \sum_j e^{s_{y_{j}}} } = \dfrac { e^{s_{y_{i}}}} { \sum_j e^{s_{y_{j}}} }$$

In [ ]:
f = np.array([123, 456, 789])
f -= f.max()
p = np.exp(f) / np.sum(np.exp(f))
print(f, p)

##### $\color{brown}{\text{Дополнительная информация}}$

**Итог**

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier_itog.png" width="800">